### Colin Alberts

## 0: Importing Packages and Defining Variables

### 0.0: Importing Packages

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import copy

import plotly.graph_objs as go
import matplotlib.pyplot as plt

### 0.1: Defining Variables

| Barrier Acronym | Barrier Name |
|-----------------|-----------------|
| UIC | Up-and-In Call |
| UIP | Up-and-In Put |
| UOC | Up-and-Out Call |
| UOP | Up-and-Out Put |
| DIC | Down-and-In Call |
| DIP | Down-and-In Put |
| DOC | Down-and-Out Call |
| DOP | Down-and-Out Put |

In [ ]:
T = 1               # expiry time
r = 0.1             # no-risk interest rate
sigma = 0.1         # volatility of underlying asset
delta = 0.0         # dividend rate
E = 100.            # exercise price
S_max = 2 * E       # upper bound of price of the stock (chosen somewhat arbitrarily, unlikely a security will double in price)
S_min = 0           # this is redundant
B = 120             # barrier price
N = int(T * 100)    # time iterations
M = int(S_max * 4)  # space iterations, accurate to about 25 cents was 4
option_type = "UOC"

# time grid
ht = T / N
ht_small = ht / 4
t_end = np.linspace(start = T - ht,
                    stop = T,
                    num = 5)
t_start = np.linspace(start = 0,
                      stop = T - ht,
                      num = N - 1,
                      endpoint = False)
t = np.concatenate((t_start, t_end))

# spatial grid (stock's price)
s, hs = np.linspace(0,
                    S_max,
                    M + 1,
                    retstep = True)


def initial_condition(x, option):
    x_copy = x
    
    if option in ["Euro Call", "UIC", "UOC", "DIC", "DOC"]:
        call_payoff = np.maximum(s - E, 0)
        
        if option in ["UIC", "DOC"]:
            call_payoff[s < B] = 0
        elif option in ["UOC", "DIC"]:
            call_payoff[s >= B] = 0
        
        x_copy[:, -1] = call_payoff
        x_copy[0, :] = 0
        x_copy[-1, :] = np.exp(-r * (T - t)) * (S_max - E)
    
    elif option in ["Euro Put", "UIP", "UOP", "DIP", "DOP"] :
        put_payoff = np.maximum(E - s, 0)
        
        if option in ["UIP", "DOP"]:
            put_payoff[s < B] = 0
        elif option in ["UOP", "DIP"]:
            put_payoff[s >= B] = 0
            
        x_copy[:, -1] = put_payoff
        x_copy[0, :] = np.exp(-r * (T - t)) * (E - S_min)
        x_copy[-1, :] = 0

    return x_copy

# solution grid
solution = np.zeros(shape = (M + 1, N + 1 + 3)) # the extra 3 is for Rannacher smoothing
solution = initial_condition(solution, option_type)

# averaging at strike to preserve second-order accuracy in Crank-Nicolson
if option_type in ["Euro Call", "Euro Put"]:
    solution[int(np.ceil(M / 2))][-1] = (solution[int(np.ceil(M / 2)) - 1][-1] + solution[int(np.ceil(M / 2)) + 1][-1]) / 2
# averaging at the barrier
elif option_type in ["UIC", "UOC", "DIC", "DOC",  "UIP", "UOP", "DIP", "DOP"]:
    if B in s:
        B_index = np.where(s == B)[0]
        solution[B_index][-1] = (solution[B_index - 1][-1] + solution[B_index + 1][-1]) / 2
    else:
        # this is more concise than writing out a binary search
        B_right_index = np.searchsorted(s, B)
        B_left_index = B_right_index - 1
        
        # have to store temporary variables since I will be using each to average the other
        temp = solution[B_right_index][-1]
        solution[B_right_index][-1] = (solution[B_right_index - 1][-1] + solution[B_right_index + 1][-1]) / 2
        solution[B_left_index][-1] = (solution[B_left_index - 1][-1] + temp) / 2

### 0.2: Optimization Functions

I used the Thomas Algorithm implementation from [this](https://github.com/katpirat/Finite-difference-for-PDEs/blob/main/FD_for_BlackScholes_pricing.py) repo.

In [ ]:
# Thomas Algorithm, is only stable when matrix is diagonally dominant or SPD.
def thomas(a, b, c, d):
    dim = len(d)  # number of equations to be solved
    ac, bc, cc, dc = map(np.array, (a, b, c, d))
    
    for i in range(1, dim):
        w = ac[i - 1] / bc[i - 1]
        bc[i] = bc[i] - w * cc[i - 1]
        dc[i] = dc[i] - w * dc[i - 1]
        
    x = bc
    x[-1] = dc[-1] / bc[-1]

    for j in range(dim - 2, -1, -1):
        x[j] = (dc[j] - cc[j] * x[j + 1]) / bc[j]
        
    return x

I check for diagonal dominance in the matrix that I solve the system with. If it is not diagonally dominant then bad things happen (convergence issues, instability, increased computation cost, etc.).

In [ ]:
# Checking diagonal dominance for Thomas Algorithm stability
def check_diag_dominant(mat):
    diags = np.abs(np.diag(mat))
    return np.all(diags >= np.sum(np.abs(mat), axis=1).T - diags)

# 1: Implementation

## 1.1: Methods

### 1.1.0: Empirical

In [ ]:
def empirical(s, sigma, r, delta, option, time):
    solution = np.zeros(shape = (s.shape[0], time.shape[0]))
    
    if option in ["Euro Call", "Euro Put"]:
        for i, current_s in enumerate(s):
            d_1 = (np.log(current_s / E) + (r - delta + sigma ** 2 / 2) * (T - time)) / (sigma * np.sqrt(T - time))
            d_2 = d_1 - sigma * np.sqrt(T - time)
            
            if option == "Euro Call":
                solution[i, :] = current_s * np.exp(-delta * (T - time)) * norm.cdf(d_1) - E * np.exp(-sigma * (T - time)) * norm.cdf(d_2)
            elif option == "Euro Put":
                solution[i, :] = E * np.exp(-sigma * (T - time)) * norm.cdf(-d_2) - current_s * np.exp(-delta * (T - time)) * norm.cdf(-d_1)
    
    return solution[:, ::-1] # flipped for plotting consistency purposes

### 1.1.1: Crank-Nicolson with Rannacher Smoothing
The [link](https://github.com/katpirat/Finite-difference-for-PDEs/blob/main/FD_for_BlackScholes_pricing.py) to the implementation that I ~~stole~~ _took inspiration_ from.

The [link](https://www.researchgate.net/publication/228524629_Convergence_analysis_of_Crank-Nicolson_and_Rannacher_time-marching) that explains why I used quarter steps for ONE iteration in the Rannacher smoothing. See the last paragraph of page 107.

Read [this](http://www.goddardconsulting.ca/option-pricing-finite-diff-crank-nicolson.html) to learn more about Crank-Nicolson differencing for Black-Scholes-Merton.

In [ ]:
def crank_smoothed(s, sigma, r, delta, solution):
    crank_solution = copy.deepcopy(solution)
    
    # Crank-Nicolson vectors
    u_vec = (sigma ** 2 * s ** 2) / (4 * hs ** 2) - (r - delta) * s / (4 * hs)
    v_vec = (sigma ** 2 * s ** 2) / (2 * hs ** 2) + r / 2
    w_vec = (sigma ** 2 * s ** 2) / (4 * hs ** 2) + (r - delta) * s / (4 * hs)
    
    P_mat_C = -np.diag(u_vec[2:-1] * ht, -1) + np.diag(v_vec[1:-1] * ht) - np.diag(w_vec[1:-2] * ht, 1) + np.identity(M - 1)
    
    offset_1 = np.zeros(M - 1)
    offset_2 = np.zeros(M - 1)
    
    # Backward Euler setup
    alpha = ht_small * (-sigma ** 2 * s ** 2 / hs ** 2 + s * (r - delta)/ hs) / 2
    beta = ht_small * (r + sigma ** 2 * s ** 2 / hs ** 2)
    gamma = ht_small * (-s ** 2 * sigma ** 2 / hs ** 2 - s * (r - delta)/ hs) / 2
    
    P_mat_E = np.diag(alpha[2:-1], -1) + np.diag(beta[1:-1]) + np.diag(gamma[1:-2], 1) + np.identity(M - 1)
    
    offset = np.zeros(M - 1)
    
    for i in range(N + 3, 0, -1):
        if i > N - 1:
            if not check_diag_dominant(P_mat_E):
                print("Matrix is NOT diagonally dominant!!!")
            
            offset[0], offset[-1] = crank_solution[0, i] * alpha[1], crank_solution[-1, i] * gamma[-1]
            
        
            crank_solution[1:-1, i - 1] = thomas(np.diag(P_mat_E, -1),
                                                 np.diag(P_mat_E),
                                                 np.diag(P_mat_E, 1),
                                                 crank_solution[1:-1, i] - offset) # the problem was multiplying 
                                                                                   # the offset by ht in initialization and here (twice)
        else:
            if not check_diag_dominant(P_mat_C):
                print("Matrix is NOT diagonally dominant!!!")
                
            offset_1[0] = crank_solution[0, i] * u_vec[1] * -2
            offset_1[-1] = crank_solution[-1, i] * w_vec[-1] * -2

            offset_2[0] = crank_solution[0, i - 1] * u_vec[1] * -2
            offset_2[-1] = crank_solution[-1, i - 1] * w_vec[-1] * -2
            
            Q_mat = np.diag(u_vec[2:-1] * ht, -1) - np.diag(v_vec[1:-1] * ht) + np.diag(w_vec[1:-2] * ht, 1) + np.identity(M - 1)
            RHS = np.dot(Q_mat, crank_solution[1:-1, i]) - ht / 2 * (offset_1 + offset_2)
                
            crank_solution[1:-1, i - 1] = thomas(np.diag(P_mat_C, -1),
                                                np.diag(P_mat_C),
                                                np.diag(P_mat_C, 1),
                                                RHS)
        
    df = pd.DataFrame(crank_solution, columns = list(map(lambda i: str(ht * i), 
                                                         range(crank_solution.shape[1]))))

    return df

### 1.1.2: Crank-Nicolson (plain)
Want this without smoothing for comparison purposes.

In [ ]:
def crank(s, sigma, r, delta, solution):
    crank_solution = copy.deepcopy(solution)
    
    # Crank-Nicolson vectors
    u_vec = (sigma ** 2 * s ** 2) / (4 * hs ** 2) - (r - delta) * s / (4 * hs)
    v_vec = (sigma ** 2 * s ** 2) / (2 * hs ** 2) + r / 2
    w_vec = (sigma ** 2 * s ** 2) / (4 * hs ** 2) + (r - delta) * s / (4 * hs)
    
    P_mat_C = -np.diag(u_vec[2:-1] * ht, -1) + np.diag(v_vec[1:-1] * ht) - np.diag(w_vec[1:-2] * ht, 1) + np.identity(M - 1)
    
    offset_1 = np.zeros(M - 1)
    offset_2 = np.zeros(M - 1)
    
    for i in range(N, 0, -1):
        if not check_diag_dominant(P_mat_C):
                print("Matrix is NOT diagonally dominant!!!")
                
        offset_1[0] = crank_solution[0, i] * u_vec[1] * -2
        offset_1[-1] = crank_solution[-1, i] * w_vec[-1] * -2

        offset_2[0] = crank_solution[0, i - 1] * u_vec[1] * -2
        offset_2[-1] = crank_solution[-1, i - 1] * w_vec[-1] * -2
            
        Q_mat = np.diag(u_vec[2:-1] * ht, -1) - np.diag(v_vec[1:-1] * ht) + np.diag(w_vec[1:-2] * ht, 1) + np.identity(M - 1)
        RHS = np.dot(Q_mat, crank_solution[1:-1, i]) - ht / 2 * (offset_1 + offset_2)
                
        crank_solution[1:-1, i - 1] = thomas(np.diag(P_mat_C, -1),
                                             np.diag(P_mat_C),
                                             np.diag(P_mat_C, 1),
                                             RHS)
        
    df = pd.DataFrame(crank_solution, columns = list(map(lambda i: str(ht * i), 
                                                         range(crank_solution.shape[1]))))

    return df

### 1.1.3: Backward Euler

In [ ]:
# def backward_euler(s, sigma, r, delta, solution):
#     crank_solution = copy.deepcopy(solution)
    
#     alpha = ht * (sigma ** 2 * s ** 2 / hs ** 2 - s / hs * (r - delta)) / 2
#     beta = ht * (r + sigma ** 2 * s ** 2 / hs ** 2)
#     gamma = ht * (s ** 2 * sigma ** 2 / hs ** 2 + s / hs * (r - delta)) / 2
    
#     P_mat = -np.diag(alpha[2:-1], -1) + np.diag(beta[1:-1]) - np.diag(gamma[1:-2], 1) + np.identity(M - 1)
    
#     offset = np.zeros(M - 1)
    
#     for i in range(N, 0, -1):
#         offset[0], offset[-1] = crank_solution[0, i] * alpha[1], crank_solution[-1, i] * gamma[-1]
        
#         crank_solution[1:-1, i - 1] = thomas(np.diag(P_mat, -1),
#                                              np.diag(P_mat),
#                                              np.diag(P_mat, 1),
#                                              crank_solution[1:-1, i] - offset)
        
#     df = pd.DataFrame(crank_solution, columns = list(map(lambda i: str(ht * i), 
#                                                          range(N + 1))))
#     return df

## 1.2: Plot Function

In [ ]:
def surface_plot(solution):
    X, Y = np.meshgrid(t[::-1], s)
    
    fig = go.Figure(data=[go.Surface(z=solution, x=Y, y=X)])
    
    fig.update_layout(title='Price Surface', 
                      autosize=False,
                      scene=dict(xaxis_title='Price',
                                 yaxis_title='Time',
                                 zaxis_title='Payoff'))
    fig.show()

## 1.3: Solve and Plot

### 1.3.1: CN with Rannacher

In [ ]:
crank_sol = crank_smoothed(s, sigma, r, delta, solution)
surface_plot(crank_sol)

In [ ]:
# something is going on with the spurious oscillations of the system
# are attemping to apply BE to make the oscillations go away, this seems to not be working
plt.plot(t, crank_sol.iloc[-2, :])
# THIS IS FIXED, KEEP THIS IN TO SHOW LATER
# This was being caused by multiplying the offset by ht twice.

### 1.3.2: CN

In [ ]:
# need to reset t and grid for following methods
t = np.linspace(start = 0,
                stop = T,
                num = N + 1)
# solution grid
solution = np.zeros(shape = (M + 1, N + 1))
solution = initial_condition(solution, option_type)

In [ ]:
plain_crank_sol = crank(s, sigma, r, delta, solution)
surface_plot(plain_crank_sol)

In [ ]:
plt.plot(t, plain_crank_sol.iloc[-2, :])

### 1.3.3: Empirical

In [ ]:
empirical_sol = empirical(s, sigma, r, delta, option_type, t[:-1])
surface_plot(empirical_sol)